# Creating Toronto neighborhoods dataframe

Install and import required libraries for web scraping and parsing

In [1]:
#!pip install beautifulsoup4
#!pip install lxml
#!pip install requests
import bs4 as bs
import requests
import pandas as pd

## Part 1: Download and explore dataset

The code below scrapes the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = bs.BeautifulSoup(source, 'lxml')
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '') for tr in td]
    data.append(row)

Transform the data into a pandas dataframe

In [3]:
df = pd.DataFrame(data, columns=["Postcode", "Borough", "Neighborhood"])

# Ignore cells with a borough that is Not assigned or Null 
df = df[df['Borough'] != 'Not assigned']
df = df[df['Borough'].notnull()]

# Combine neighborhoods into one comma separated row
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].agg(lambda x : ','.join(x)).to_frame().reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough (only 1 case)
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = 'Queen\'s Park'

In [4]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df.shape

(103, 3)

## Part 2: get coordinates for each neighborhood

Download dataset with geolocations

In [6]:
!wget -q -O 'geospacial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
geospacial_data = pd.read_csv('geospacial_data.csv')
geospacial_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)
geospacial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine two dataframes

In [8]:
df = df.merge(geospacial_data, on='Postcode')
df.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Part 3: Toronto neighborhoods analysis

In [14]:
#!pip install folium
#!pip install geopy
import folium # map rendering library
from geopy.geocoders import Nominatim 

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto neighborhoods

In [18]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map